# FastText
<h4 style="font-size:14px; font-family:Calibry" align="left"> Andrii Kruchko </h4>

<a id='0'> </a>

<hr style="height: 1px; background-color: #808080">
## Table of Contents

<ol>
    <li style="font-size:20px; font-family:Verdana">[FastText overview](#1)</li>
    <li style="font-size:20px; font-family:Verdana">[Data preprocessing for fastText](#2)</li>
    <li style="font-size:20px; font-family:Verdana">[Parameters overview and the model training](#3)</li>
</ol>

<a id='1'></a>

<hr style="height: 1px; background-color: #808080">
## 1. FastText overview
### What is it?

FastText is a linear model with a rank constraint and a fast loss approximation.<br>
It can obtain the accuracy comparable to deep learning classifiers.<br>

But it is way faster:
- FastText can train on more than one 200M words in less than five minutes using a standard multicore CPU
- Classify nearly 150K reviews in less than a minute

### Architecture

<img src="https://raw.githubusercontent.com/akruchko/test/master/1_model_architecture_of_fastText.PNG">
The model architecture of fastText for a sentence with N ngram features x1, . . . , xN .<br> The features are embedded and averaged to form the hidden variable$^1$

<hr style="height: 1px; width: 100px; background-color: #808080"; align="left"> <br>
$^1$ https://arxiv.org/pdf/1607.01759.pdf

### Algorithm

FastText uses the softmax function $f$ to compute the probability distribution over the predefined classes. For a set of N documents, this leads to minimizing the negative loglikelihood over the classes:


\begin{align}
\ -\frac{1}{N} \sum_{n=1}^N y_n log(f(BAx_n))
\end{align}
$x_n$ - the normalized bag of features of the n-th document, <br>
$y_n$ - the label, <br>
$A, B$ - weight matrices

Optimization is performing using stochastic gradient descent and a linearly decaying learning rate.

<h3 style="font-size:16px; font-family:Verdana">[To the table of contents](#0)</h3>

<a id='2'></a>

<hr style="height: 1px; background-color: #808080">
## 2. Data preprocessing for fastText
- remove nonprintable characters
- fix $n't$, $'re$, $'s$ and other cases
- remove punctuation and digits
- Porter stemming

In [1]:
import pandas as pd

from itertools import product

from string import punctuation, digits
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
from fasttext import supervised

In [3]:
df_train = pd.read_csv('data/movie_reviews.csv')
df_test = pd.read_csv('data/test.csv')

In [4]:
# prepare punctuation and digits list for removal
translator = str.maketrans('', '', punctuation + digits)

# basic preprocessing:
def clean_data(df, col):
    df['clean_text'] = df[col].str.replace('\n', '').str.replace('\r', '').str.replace('\t', '')
    df.clean_text = df.clean_text.str.replace("n't", " not").str.replace("'re", " are").str.replace("'s", " s")
    df.clean_text = df.clean_text.str.replace("'ve", " have").str.replace("'ll", " will").str.replace("'d", " d")
    df.clean_text = df.clean_text.str.translate(translator).str.strip().str.lower()
    return df

In [5]:
df_train = clean_data(df_train, 'text')
df_test = clean_data(df_test, 'text')

In [6]:
df_train.head(1)

,label,text,clean_text
0,1,"To an entire generation of filmgoers, it just ...",to an entire generation of filmgoers it just m...


In [7]:
%%time
# Porter stemming
stemmer = PorterStemmer()
df_train['porter_text'] = df_train['clean_text'].apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split()]))
df_train = df_train[df_train.porter_text.apply(len) != 0]
df_test['porter_text'] = df_test['clean_text'].apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split()]))

CPU times: user 2min 30s, sys: 68 ms, total: 2min 31s
Wall time: 2min 31s


In [8]:
df_train.head(1)

,label,text,clean_text,porter_text
0,1,"To an entire generation of filmgoers, it just ...",to an entire generation of filmgoers it just m...,to an entir gener of filmgoer it just might re...


In [9]:
# splitting on train and validation
df_train2, df_val = train_test_split(df_train[['label', 'porter_text']], test_size=0.2, random_state=42)

In [10]:
# Since fastText can be trained only from text files, we should mark labels. The default is `__label__` but can be custom.
df_train2['ft_label'] = df_train2['label'].apply(lambda x: '__label__1 ' if x == 1 else '__label__0 ')
df_train2[['ft_label', 'porter_text']].to_csv('data/train_fastText.csv', index=False, header=False)

### observations and yet another preprocessing variant

Weren't helpful:
- default stop words make the model performance worse. The list should be revised or created from scratch
- entities removal influences model's results badly. 

SpaCy lemmatization less aggressive than Porter stemmer

<h3 style="font-size:16px; font-family:Verdana">[To the table of contents](#0)</h3>

<a id='3'></a>

<hr style="height: 1px; background-color: #808080">
## 3. Parameters overview and the model training

### Parameters overview

- `lr` - learning rate. Default: **0.1**.
- `dim` - size of word vectors in the hidden unit. Default: **100**. Should be less for small datasets and the number of labels.
- `epoch` - number of epochs. Default: **5**. Higher for small learning rates.
- `min_count` - minimal number of word occurences. Default: **1**. 5 or higher to avoid overfitting.
- `word_ngrams` - max length of word ngram. Default: **1**. Higher order ngrams lead to overfitting on small datasets. if value greater than 1 learning rate and epoch should be revised.
- `bucket` - number of buckets. Default: **2000000**. Developers recommend to use lower values for small datasets (ex. 100K).
- `minn` - min length of char ngram. Default: **0**.
- `maxn` - max length of char ngram. Default: **0**.

### The model training

In [12]:
def get_score(df, clf, label='label', text='porter_text', model_name='data/fastText_porter'):
  
    prediction = clf.predict_proba(list(df[text]))
    prediction = [int(item[0][0]) for item in prediction]

    return [round(accuracy_score(list(df[label]), prediction), 4), 
            round(f1_score(list(df[label]), prediction, pos_label=0), 4)]

In [13]:
# prepare parameters for the grid search
lrates = [0.5, 0.1, 0.01]
epochs = [5, 10]
min_c = [1, 5]

In [14]:
%%time
# let's train models and evaluate them on the validation dataset
results = []
for i in product(lrates, epochs, min_c):
    clf = supervised('data/train_fastText.csv', 'data/fastText_porter', label_prefix='__label__', 
                     lr=i[0], epoch=i[1], min_count=i[2])    
    results.append(list(i) + get_score(df_val, clf))

CPU times: user 6min 44s, sys: 11.1 s, total: 6min 56s
Wall time: 4min 3s


In [15]:
results = pd.DataFrame(results, columns=['lr', 'epoch', 'min_count', 'accuracy', 'f1']).sort_values(by='f1', ascending=False)
results.reset_index(drop=True, inplace=True)
results

,lr,epoch,min_count,accuracy,f1
0,0.50,5,5,0.8163,0.7726
1,0.10,5,5,0.8170,0.7724
2,0.10,10,5,0.8159,0.7719
3,0.50,5,1,0.8176,0.7702
4,0.10,5,1,0.8168,0.7691
5,0.10,10,1,0.8162,0.7691
6,0.50,10,5,0.8159,0.7687
7,0.50,10,1,0.8146,0.7669
8,0.01,10,5,0.8143,0.7654
9,0.01,10,1,0.8139,0.7641


In [16]:
clf = supervised('data/train_fastText.csv', 'data/fastText_porter', label_prefix='__label__', 
                 lr=results.lr[0], epoch=results.epoch[0], min_count=results.min_count[0])

In [17]:
acc, f1 = get_score(df_test, clf)
print('test accuracy:', round(acc, 4), 'test f1:', round(f1, 4))

test accuracy: 0.7869 test f1: 0.7697


### The exercise # 1
Train models with predefined parameters and explain why such results were derived.

In [ ]:
%%time
# A
clf = supervised('data/train_fastText.csv', 'data/fastText_porter', label_prefix='__label__', 
                 epoch=40)

In [ ]:
acc, f1 = get_score(df_val, clf)
print('validation accuracy:', round(acc, 4), 'validation f1:', round(f1, 4))
acc, f1 = get_score(df_test, clf)
print('test accuracy:', round(acc, 4), 'test f1:', round(f1, 4))

In [ ]:
%%time
# B
clf = supervised('data/train_fastText.csv', 'data/fastText_porter', label_prefix='__label__', 
                 word_ngrams=3, 
                 bucket=2000000)

In [ ]:
acc, f1 = get_score(df_val, clf)
print('validation accuracy:', round(acc, 4), 'validation f1:', round(f1, 4))
acc, f1 = get_score(df_test, clf)
print('test accuracy:', round(acc, 4), 'test f1:', round(f1, 4))

### The exercise # 2
Try to improve the previous results using diiferent values of parameters.

In [ ]:
%%time
clf = supervised('data/train_fastText.csv', 'data/fastText_porter', label_prefix='__label__',
                 lr=0.1, 
                 dim=100, 
                 epoch=5, 
                 min_count=1, 
                 word_ngrams=1, 
                 bucket=2000000, 
                 minn=0, 
                 maxn=0
                )

In [ ]:
acc, f1 = get_score(df_val, clf)
print('validation accuracy:', round(acc, 4), 'validation f1:', round(f1, 4))
acc, f1 = get_score(df_test, clf)
print('test accuracy:', round(acc, 4), 'test f1:', round(f1, 4))

<h3 style="font-size:16px; font-family:Verdana">[To the table of contents](#0)</h3>

## Conclusions

- Really fast: representing sentences with bag of words and bag of n-grams with hashing trick; a hierachical softmax
- It was developed mainly for large datasets (ex. 1 billion words). In case of small datasets hyperparameters should be tuned carefully to avoid overfitting or you shoud get more data.